## Config

In [ ]:
!pip install lifelines
!pip install gspread

In [ ]:
!pip install gspreadfrom dateutil.relativedelta import relativedelta
from datetime import datetime, timedelta, date
import pandas as pd
from google.cloud import bigquery
import random
import matplotlib.pyplot as plt
from lifelines import KaplanMeierFitter
import numpy as np
from scipy.optimize import curve_fit
from google.colab import auth

In [ ]:
auth.authenticate_user()

In [ ]:
client = bigquery.Client(project='indy-eng')

In [ ]:
# This map is needed to translate trial_duration into trial_duration in months
cadence_map = {
    "day": 1,
    "Staging test": 0,
    "week": 1,
    "2 week": 1,
    "month": 1,
    "quarter": 3,
    "6 month":  6,
    "year": 12,
    "3 year": 36,
    "No trial": 0
}

# Retention Curves

## Creating Retention Curves

In [ ]:
# Dimensions used to create categorisation of our new retention curves
splits = [
    "package_type",
    "trial_duration_months",
    "term_cadence",
]

splits_str = ", ".join(splits)

In [ ]:
subs_details_rundate = "2025-10-17"


In [ ]:
query = f"""

WITH fixed_expiry_date_df AS (
    SELECT * EXCEPT(user_access_expiration_date),
    CASE WHEN LOWER(piano_status) LIKE 'payment failure'
    # Move the expiry date up by 28 days for payment failures as these individuals will not have payed for their last 28 days of the subscription
    OR (LOWER(piano_status) LIKE 'active' AND subscription_grace_period_start_date IS NOT NULL AND RAND() < 0.8)
    THEN DATE_SUB(user_access_expiration_date, INTERVAL 28 DAY)
    ELSE user_access_expiration_date END AS fixed_expiry_date
    FROM
    `indy-eng.reader_revenue_dataform.subscription_details_snapshots_enhanced`
    WHERE
        run_date = "{subs_details_rundate}"
)

SELECT

    {splits_str},

    CAST(
        CASE
            WHEN CAST(fixed_expiry_date AS DATE) > "{subs_details_rundate}"
            #NOTE SOME USERS DO NOT SEEM TO HAVE AN EXPIRY DATE - DYNAMIC ISSUE, IN ANY CASE STILL NEED TO BE ADDED
            OR fixed_expiry_date IS NULL
            THEN DATE_DIFF("{subs_details_rundate}", CAST(start_date AS DATE), MONTH)
            ELSE
                DATE_DIFF(CAST(fixed_expiry_date AS DATE), CAST(start_date AS DATE), MONTH)
        END AS FLOAT64
    ) AS duration,

    CASE
        WHEN CAST(fixed_expiry_date AS DATE) < "{subs_details_rundate}" THEN 1
        ELSE 0
    END AS churned

FROM
    fixed_expiry_date_df

WHERE
  customer_type NOT LIKE "Test"
  AND LOWER(package_type) NOT LIKE "%app%"
  AND customer_type NOT LIKE "Membership"
  AND start_date IS NOT NULL
"""

query_job = client.query(query)
retention_curves_df = query_job.to_dataframe()

retention_curves_df["trial_duration_months"].fillna(0, inplace=True)

In [ ]:
# Fitting the retention curves, to be cleaned w/ Alex's version of the split
def naive_forecast(survival_df, months_to_extend, plot=False):
    """
    Extends a survival dataframe using a naïve forecast where future retention rates
    remain constant at the last observed value.

    Parameters
    ----------
    survival_df : pd.DataFrame
        DataFrame with columns ['month_index', 'retention_curve_rate'].
    months_to_extend : int
        Number of months to extend the forecast.
    plot : bool, optional
        Whether to plot the historical and forecasted retention curves.

    Returns
    -------
    pd.DataFrame
        Combined DataFrame with both historical and forecasted retention rates.
    """

    last_month = survival_df["month_index"].max()
    last_value = survival_df["retention_curve_rate"].iloc[-1]

    future_months = pd.DataFrame({
        "month_index": np.arange(last_month + 1, last_month + 1 + months_to_extend),
        "retention_curve_rate": last_value
    })

    forecasted_df = pd.concat([survival_df, future_months], ignore_index=True)

    if plot:
        plt.figure(figsize=(8, 4))
        plt.plot(
            survival_df["month_index"],
            survival_df["retention_curve_rate"],
            label="Historical",
            marker="o",
        )
        plt.plot(
            future_months["month_index"],
            future_months["retention_curve_rate"],
            label="Forecast",
            linestyle="--",
            color="orange",
            marker="o",
        )
        plt.title("Naïve Retention Forecast")
        plt.xlabel("Month Index")
        plt.ylabel("Retention Curve Rate")
        plt.legend()
        plt.grid(True, alpha=0.3)
        plt.tight_layout()
        plt.show()

    return forecasted_df


def extend_retention_curve(survival_df: pd.DataFrame, trial_months: int, term_cadence: str, months_to_extend: int = 27) -> pd.DataFrame:
    """
    Extend the Kaplan–Meier survival curve by repeating the last observed value
    for a specified number of months.

    Parameters
    ----------
    survival_df : pd.DataFrame
        DataFrame with columns ["month_index", "percentage_of_users"].
    months_to_extend : int, default=12
        Number of additional months to extend the curve.

    Returns
    -------
    pd.DataFrame
        Extended survival DataFrame including the additional months.
    """

    last_month = survival_df["month_index"].max()
    eligible_forecast_months = last_month - trial_months

    tenured_months_df = survival_df[survival_df["month_index"] > trial_months]

    required_periods_to_forecast = 4

    forecast_df = naive_forecast(survival_df, months_to_extend)

    return forecast_df

def apply_km(group: pd.DataFrame) -> pd.DataFrame:
    """
    Fit a Kaplan–Meier survival curve for a given group and extend it

    Parameters
    ----------
    group : pd.DataFrame
        Input data containing 'duration' and 'churned' columns, plus cohort splits.

    Returns
    -------
    pd.DataFrame
        Extended survival DataFrame with cohort identifiers.
    """

    trial_months = group["trial_duration_months"].iloc[0]
    term_cadence = group["term_cadence"].iloc[0]

    kmf = KaplanMeierFitter()

    durations = group["duration"]
    events = group["churned"].astype(int)  # 1 = churned, 0 = censored

    timeline = np.arange(0, group["duration"].max() + 1)

    kmf.fit(durations, event_observed=events, timeline=timeline)

    survival_df = kmf.survival_function_.reset_index()
    survival_df.columns = ["month_index", "retention_curve_rate"]

    # Extend retention curve
    extended_survival_df = extend_retention_curve(survival_df, trial_months=trial_months, term_cadence=term_cadence, months_to_extend=24)

    # Calculating Piecewise retention Curve
    extended_survival_df["piecewise_retention_rate"] = extended_survival_df["retention_curve_rate"] / extended_survival_df["retention_curve_rate"].shift(1)

    for split in splits:
        extended_survival_df[split] = group.iloc[0][split]

    return extended_survival_df


retention_curves = retention_curves_df.groupby(
    splits,
    group_keys=False
).apply(apply_km)

In [ ]:
checking_map = {'package_type': 'DIGITAL Subscriber', 'trial_duration_months': 6, 'term_cadence': 'year'}
retention_curves[
    (retention_curves['package_type'] == checking_map['package_type']) &
    (retention_curves['trial_duration_months'] == checking_map['trial_duration_months']) &
    (retention_curves['term_cadence'] == checking_map['term_cadence'])
]

## Retention Curves Testing

In [ ]:
if retention_curves.isna().sum().sum() > 0:
    print(retention_curves.isna().sum())

    retention_curves[retention_curves.piecewise_retention_rate.isna()]

### Visualising Retention Curves

In [ ]:
def plot_km(**kwargs):

    if set(kwargs.keys()) != set(splits):
        raise ValueError(f"Invalid filter keys: {list(kwargs.keys())}. Keys must be: {splits}")

    subset_df = retention_curves.copy()

    for col, value in kwargs.items():
        subset_df = subset_df[subset_df[col] == value]

    label = " - ".join(str(v) for v in kwargs.values())

    plt.plot(
        subset_df["month_index"],
        subset_df["retention_curve_rate"],
        linestyle="-",
        label=label,
    )

for i in range(15):

    random_row = retention_curves[splits].sample(1).iloc[0]
    random_splits = random_row.to_dict()

    print(random_splits)

    plt.figure(figsize=(8, 5))
    plt.xlabel("Month Index")
    plt.ylabel("Retention Rate")
    plt.grid(True)
    plt.tight_layout()

    plt.title("Retention Curves")

    plot_km(
        **random_splits
    )

    plt.legend()
    plt.show()

# User Bases

In [ ]:
base_splits = splits + ["trial_price_value","trial_price","term_price", "term_price_value", "payment_currency", "region","signup_cohort", "trial_duration"]
base_splits_str = ", ".join(base_splits)

start_date = '2015-01-01'
latest_date = '2025-09-01'
forecast_to_date = '2027-12-01'

# Based on 6-month average (March-July 2025)
payment_failure_loss_rate = 0.8

query = f"""
WITH
# Creating calendar dates to fill in missing months + add future months
  calendar AS (
    SELECT calendar_month
    FROM UNNEST(GENERATE_DATE_ARRAY('{start_date}', '{forecast_to_date}', INTERVAL 1 MONTH)) AS calendar_month
  ),

  subs_details AS (
  SELECT
# Base splits are all the different cohorts in our user base: This includes the retention curves splits and the sign up cohorts
    {base_splits_str},

    calendar_month,
#  This pulls expiration date back 28 days for (potential) payment failures as these individuals will not make any payments on their last 28 days (this makes forecasting revenue more accurate)
    DATE(
      DATE_TRUNC(
        CASE
          WHEN (LOWER(piano_status) LIKE 'payment failure') OR (LOWER(piano_status) LIKE 'active' AND subscription_grace_period_start_date IS NOT NULL AND RAND() < {payment_failure_loss_rate})
            THEN DATE_SUB(user_access_expiration_date, INTERVAL 28 DAY)
          ELSE
            user_access_expiration_date
        END
      , MONTH))
    AS expire_month,

    DATE(DATE_TRUNC(subscription_trial_end_date, MONTH)) AS trial_end_month,

  FROM
    `indy-eng.reader_revenue_dataform.subscription_details_snapshots_enhanced`
  JOIN
    calendar ON calendar.calendar_month >= DATE(DATE_TRUNC(start_date, MONTH))
# Membership is removed here as it is included in the final forecast doc
  WHERE
     customer_type NOT LIKE "Test"
  AND LOWER(package_type) NOT LIKE "%app%"
  AND customer_type NOT LIKE "Membership"
  AND start_date IS NOT NULL
  AND is_blacklisted IS FALSE
  AND run_date = "{subs_details_rundate}"
  )

SELECT

  {base_splits_str},

  calendar_month,
  DATE_DIFF(calendar_month, signup_cohort, month) AS month_index,

  SUM(
    CASE
        WHEN
            (calendar_month BETWEEN signup_cohort AND (expire_month - 1))
            # OR (expire_month IS NULL)
        THEN 1
    ELSE 0 END) AS active_users,

  MAX(COUNT(*)) OVER(PARTITION BY {base_splits_str}) as total_cohort_users

FROM
  subs_details
WHERE
  signup_cohort <= '{latest_date}'

GROUP BY
    ALL
ORDER BY
    signup_cohort, calendar_month
"""

query_job = client.query(query)
cohort_df = query_job.to_dataframe()

In [ ]:
# Filling in any empty entries w/ null values
cohort_df["trial_duration_months"].fillna(0, inplace=True)
cohort_df["trial_price_value"].fillna(0, inplace=True)

In [ ]:
# Check for NA values
mask = cohort_df["calendar_month"] >= date(2025, 11, 1)
cohort_df.loc[mask, "active_users"] = 0

In [ ]:
cohort_df.isna().sum()

In [ ]:
# Checking historical base
mask = (cohort_df["calendar_month"] > date(2025,7,1))

cohort_df[mask].groupby(["calendar_month"])["active_users"].sum().reset_index()

# Acquisition


## Calculating Subscription Acquisitions (Based on PAVs)

In [ ]:
#  Introducing CVR of different acquisition sources
# Uploading GSheet with CVR (Ideally would pull this directly from Adobe/ GA4 but nature of conversion data complicates this)

import gspread
from google.auth import default

auth.authenticate_user()
creds, _ = default()

# Authorize gspread
gc = gspread.authorize(creds)

# Opening GSheet
sheet_url = 'https://docs.google.com/spreadsheets/d/14bVpG_hj_YrBmHdCTTfckKHAkBOAlVSMeKUgEyzZqSk/edit?usp=sharing'

acq_spreadsheet = gc.open_by_url(sheet_url)

conversions_by_source = acq_spreadsheet.sheet1


# Use get_worksheet on the spreadsheet object to get the second worksheet (index 1)
cvr_by_source = acq_spreadsheet.get_worksheet(1)

#  Turning into dataframes
conv_by_source_df = pd.DataFrame(conversions_by_source.get_all_records())
cvr_by_source_df = pd.DataFrame(cvr_by_source.get_all_records())

In [ ]:
# Introducing Emily's traffic assumptions


sheet_url = 'https://docs.google.com/spreadsheets/d/1u1luHB9rRBj9ih0x9e2kBF2CA9Dj8RjRaSkvf1jZc9Q/edit?usp=sharing'

traffic_forecast = gc.open_by_url(sheet_url)

# Note: Only UK+ROW subscriptions will be tied to traffic, US traffic has been removed from the below
PAV_forecast_df = pd.DataFrame(traffic_forecast.get_worksheet(2).get_all_records())
PAV_forecast_df = PAV_forecast_df[pd.to_datetime(PAV_forecast_df['date']).dt.date> date(2025,12,1)]

HPPU_forecast_df = pd.DataFrame(traffic_forecast.get_worksheet(4).get_all_records())
HPPU_forecast_df = HPPU_forecast_df[pd.to_datetime(HPPU_forecast_df['date']).dt.date> date(2025,12,1)]

In [ ]:
#Defining subscription source
PAV_forecast_df['Subscription Experience']='Premium Article Gate'
HPPU_forecast_df['Subscription Experience']='HPPU / Section PU'

# NOTE: This adds all traffic UK+ROW into UK region - change if possible
PAV_forecast_df['region']='UK'
HPPU_forecast_df['region']='UK'

# Merging CVR into traffic Forecasts
PAV_forecast_df = pd.merge(PAV_forecast_df, cvr_by_source_df[['Subscription Experience', 'CVR']], on='Subscription Experience', how='left')
HPPU_forecast_df = pd.merge(HPPU_forecast_df, cvr_by_source_df[['Subscription Experience', 'CVR']], on='Subscription Experience', how='left')

In [ ]:
# Calculate new subscribers from PAV forecasts
PAV_forecast_df['New Subscribers'] = PAV_forecast_df['PAVs'] * PAV_forecast_df['CVR'].str.rstrip('%').astype('float') / 100.0

# Calculate new subscribers from HPPU forecasts
HPPU_forecast_df['New Subscribers'] = HPPU_forecast_df['UK_PVs'] * HPPU_forecast_df['CVR'].str.rstrip('%').astype('float') / 100.0

print("PAV Forecast:")
display(PAV_forecast_df.head())


print("\nHPPU Forecast:")
display(HPPU_forecast_df.head())


In [ ]:
# Grouping HPPU and PAG acquisitions together
forecasted_cohorts = pd.concat([
    PAV_forecast_df[['date', 'New Subscribers','Subscription Experience']],
    HPPU_forecast_df[['date', 'New Subscribers','Subscription Experience']],
], ignore_index=True)

# Introducing remaining subscription cohorts
sources = ['Navigation', 'Direct']
months_2026 = pd.date_range(start='2026-01-01', end='2026-12-01', freq='MS')

new_data = []
for source in sources:
    for month in months_2026:
        new_data.append({
            'date': month,
            'New Subscribers': 0,
            'Subscription Experience': source
        })

new_sources_df = pd.DataFrame(new_data)

forecasted_cohorts = pd.concat([forecasted_cohorts, new_sources_df], ignore_index=True)
forecasted_cohorts.date = pd.to_datetime(forecasted_cohorts.date).dt.date

#  Adding % of conversions from each sub journey

forecasted_cohorts = forecasted_cohorts.merge(conv_by_source_df[['Subscription Experience', '%']], on='Subscription Experience', how='left')

In [ ]:
# Calculate number of acquisitions for Navigation and Direct based on remaining acquisition %
forecasted_navigation=forecasted_cohorts.loc[forecasted_cohorts['Subscription Experience'] == 'Navigation', 'New Subscribers'] = (
    forecasted_cohorts[forecasted_cohorts['Subscription Experience'] == 'Premium Article Gate']['New Subscribers'].values[0] *
    forecasted_cohorts[forecasted_cohorts['Subscription Experience'] == 'Navigation']['%'].str.rstrip('%').astype('float') / 100.0 /
    forecasted_cohorts[forecasted_cohorts['Subscription Experience'] == 'Premium Article Gate']['%'].str.rstrip('%').astype('float') * 100.0
)

In [ ]:

PAG_metrics = forecasted_cohorts[forecasted_cohorts['Subscription Experience'] == 'Premium Article Gate'][['date', 'New Subscribers', '%']].copy().rename(columns={'%': 'PAG_%', 'New Subscribers': 'PAG_New_Subscribers'})
forecasted_cohorts = forecasted_cohorts.merge(PAG_metrics, on='date', how='left')

#  Calculating new acquisitions from Navigation
forecasted_cohorts.loc[forecasted_cohorts['Subscription Experience'] == 'Navigation', 'New Subscribers'] = (
    forecasted_cohorts['PAG_New_Subscribers'] *
    (forecasted_cohorts['%'].str.rstrip('%').astype('float') / 100.0 )/
    (forecasted_cohorts['PAG_%'].str.rstrip('%').astype('float') / 100.0)
)
#  Calculating new acquisitions from Direct
forecasted_cohorts.loc[forecasted_cohorts['Subscription Experience'] == 'Direct', 'New Subscribers'] = (
    forecasted_cohorts['PAG_New_Subscribers'] *
    (forecasted_cohorts['%'].str.rstrip('%').astype('float') / 100.0 )/
    (forecasted_cohorts['PAG_%'].str.rstrip('%').astype('float') / 100.0)
    )


In [ ]:
forecasted_subs = forecasted_cohorts.groupby('date')['New Subscribers'].sum()

In [ ]:
forecasted_subs = pd.DataFrame(forecasted_subs).reset_index()

In [ ]:
six_months_ago = (pd.Timestamp.now() - pd.DateOffset(months=6)).replace(day=1).date()
current_users_breakdown = cohort_df[((
    (cohort_df['package_type'].isin(['DIGITAL Subscriber']))
&   (cohort_df['term_cadence'].isin(['month', 'year']))
&   (cohort_df['term_price'].isin(['GBP11.00','EUR11.00', 'GBP99.00','EUR99.00', 'GBP4.00', 'EUR4.00']))
& (cohort_df['trial_price'].isin(['GBP1.00','EUR1.00']))
& (cohort_df['trial_duration'].isin(['6 month']))
 | (
     (cohort_df['package_type'].isin(['Consent Or Pay Only Subscriber','Student Subscription']))
&   (cohort_df['term_cadence'].isin(['month']))
&   (cohort_df['term_price'].isin(['GBP4.00','EUR4.00', 'GBP1.00','EUR1.00']))
& (cohort_df['trial_price'].isin(['No trial']))
))
& (cohort_df['signup_cohort'] >= six_months_ago)
& (cohort_df['month_index'] == 0))

]

In [ ]:
current_users_breakdown = current_users_breakdown.groupby(['region', 'package_type','trial_duration', 'term_cadence','trial_price','term_price','month_index'])['total_cohort_users'].sum().reset_index()
current_users_breakdown['all_in_cohort'] = current_users_breakdown['total_cohort_users'].sum()

In [ ]:
current_users_breakdown['perc_subs'] = current_users_breakdown['total_cohort_users'] / current_users_breakdown['all_in_cohort']

In [ ]:
combined = forecasted_subs.merge(current_users_breakdown[['region', 'package_type', 'term_cadence', 'trial_duration', 'trial_price', 'term_price', 'month_index','perc_subs']], how='cross')

In [ ]:
combined['forecasted_subs'] = combined['New Subscribers'] * combined['perc_subs']

In [ ]:
# Manually fixing erroneous term

broken_term = (
    (combined["region"] == "EUR")
    & (combined["term_cadence"] == "month")
    & (combined["term_price"] == "GBP99.00")
)
combined.loc[broken_term, 'forecasted_subs'] = 0

## Calculating recurring donation acquisition, students and COP (Based on 6-month average)

In [ ]:
acq_forecast_cohort_df =cohort_df[cohort_df['calendar_month'] < date(2025,10,1)].copy()

#  Adding unique keys for retention curve splits and cohort splits
retention_curve_splits = [split for split in splits if split != "trial_duration_months"]
added_splits = [
        'region', 'term_price', 'trial_price', 'trial_duration'
    ]



acq_forecast_cohort_df['retention_curve_splits'] = acq_forecast_cohort_df[retention_curve_splits].apply(lambda row: ','.join(row.values.astype(str)), axis=1)
acq_forecast_cohort_df['cohort_splits'] = acq_forecast_cohort_df[retention_curve_splits + added_splits].apply(lambda row: ','.join(row.values.astype(str)), axis=1)

In [ ]:
# Introducing new cohorts that will be acquired in 2026
def generate_new_cohorts(cohorts_df, forecast_months=27):
    cohorts_df = cohorts_df.copy()
    cohorts_df['calendar_month'] = pd.PeriodIndex(cohorts_df['calendar_month'], freq='M')


    acq_weights = np.array([0.083, 0.083, 0.084, 0.25, 0.25, 0.25])
    acq_weights = acq_weights / acq_weights.sum()

    future_cohorts = []

    for month_ahead in range(1, forecast_months + 1):
        last_calendar = cohorts_df['calendar_month'].max()  # global last month

        new_month = last_calendar + 1  # next calendar month

        # Process each cohort_split separately
        for cohort_splits, group in cohorts_df[cohorts_df['month_index'] == 0].groupby('cohort_splits'):

            full_range = pd.period_range(
                start=group['calendar_month'].min(),
                end=last_calendar,
                freq='M'
            )

            group = (
                group.set_index('calendar_month')
                     .reindex(full_range)
                     .rename_axis('calendar_month')
                     .reset_index()
            )
            group['total_cohort_users'] = group['total_cohort_users'].fillna(0)
            group['active_users'] = group['active_users'].fillna(0)
            for col in splits + added_splits:
                if col in group.columns:
                    group[col] = group[col].ffill().bfill()

            recent = group.sort_values('calendar_month').tail(6)
            # As some terms have been removed in Aug 2025, I have shortened length of months until we consider a term expired to 2
            is_term_active = group.sort_values('calendar_month').tail(2)
            if len(is_term_active) == 0:
                continue

            w = acq_weights[-len(recent):]
            avg_new_users = round(np.average(recent['total_cohort_users'], weights=w),0)

            new_cohort = {
                'signup_cohort': new_month,
                'calendar_month': new_month,
                'month_index': 0,
                'total_cohort_users': avg_new_users,
                'active_users': avg_new_users,
                'cohort_splits': cohort_splits
            }

            future_cohorts.append(new_cohort)

            # Append immediately so next iteration uses this new cohort
            cohorts_df = pd.concat([cohorts_df, pd.DataFrame([new_cohort])], ignore_index=True)

    return pd.DataFrame(future_cohorts)

future_cohorts = generate_new_cohorts(acq_forecast_cohort_df.copy()).sort_values(['cohort_splits','signup_cohort','calendar_month'])



In [ ]:
future_cohorts[['package_type', 'term_cadence','region', 'term_price', 'trial_price', 'trial_duration']] = future_cohorts['cohort_splits'].str.split(',', expand=True)
future_cohorts['retention_curve_splits'] = future_cohorts[retention_curve_splits].apply(lambda row: ','.join(row.values.astype(str)), axis=1)

future_cohorts['signup_cohort'] = future_cohorts['signup_cohort'].astype(str) + '-01'

future_cohorts['calendar_month'] = future_cohorts['calendar_month'].astype(str)+ '-01'


#  Set all future cohorts to be non-trialists except current monthly and current annual
future_cohorts['trial_status']='non-trialist'

In [ ]:
active_subs_terms =  (
    (future_cohorts['package_type']== 'DIGITAL Subscriber')
    & (future_cohorts['term_cadence'].isin(['month', 'year']))
    & (future_cohorts['term_price'].isin(['GBP11.00','EUR11.00', 'GBP99.00', 'EUR99.00']))
    & (future_cohorts['trial_price'].isin(['GBP1.00','EUR1.00']))
    & (future_cohorts['trial_duration']=='6 month')
)

future_cohorts.loc[active_subs_terms, 'trial_status'] = 'trialist'

In [ ]:
future_cohorts[future_cohorts['trial_status'] == 'trialist']['cohort_splits'].unique()

## Combining both methodologies (prioritising PAV methodology for subs)


In [ ]:
future_cohorts[future_cohorts['trial_status'] == 'trialist'].head(1)

In [ ]:
final_forecasted_subs = combined[['date', 'region', 'package_type', 'term_cadence', 'trial_duration', 'trial_price', 'term_price', 'month_index', 'forecasted_subs']]
final_forecasted_subs['cohort_splits'] = final_forecasted_subs[retention_curve_splits + added_splits].apply(lambda row: ','.join(row.values.astype(str)), axis=1)

In [ ]:
# Ensure key columns are same type
future_cohorts['cohort_splits'] = future_cohorts['cohort_splits'].astype(str)
final_forecasted_subs['cohort_splits'] = final_forecasted_subs['cohort_splits'].astype(str)

final_forecasted_subs['date'] = final_forecasted_subs['date'].astype(str)

merged = future_cohorts.merge(
    final_forecasted_subs[['date', 'cohort_splits', 'forecasted_subs']],
    left_on=['calendar_month', 'cohort_splits'],
    right_on=['date', 'cohort_splits'],
    how='left'
)

merged.loc[merged['forecasted_subs'].notna(), 'active_users'] = merged['forecasted_subs']
merged.loc[merged['forecasted_subs'].notna(), 'total_cohort_users'] = merged['forecasted_subs']

In [ ]:
acquisition_df = merged.copy()

acquisition_df.drop(
    [
        'retention_curve_splits',
        'cohort_splits',
        'date',
        'forecasted_subs',
        'trial_status'
    ],
    axis=1,
    inplace=True
)

future_final_cohorts = acquisition_df[pd.to_datetime(acquisition_df["calendar_month"]).dt.date >= date(2025,10,1)]

In [ ]:
future_final_cohorts

## Extending Acquisition

In [ ]:
forecast_date = date(2027,12,1)

extend_aquisition_splits = ["signup_cohort", "package_type", "term_cadence", "region", "term_price", "trial_price", "trial_duration", "total_cohort_users"]

def extend_aquisition_data(group):

    if len(group) > 1:
        raise Exception()

    start_month = group["calendar_month"].iloc[0]

    date_range = pd.date_range(start_month, forecast_date, freq="MS")

    group = group.set_index("calendar_month")
    group.index = pd.to_datetime(group.index)
    group = group.reindex(date_range)

    group['month_index'] = range(0, len(group))
    group['active_users'] = group['active_users'].fillna(0)

    group = group.reset_index().rename(columns={'index': 'calendar_month'})

    return group

extended_aquisition_data_extended = future_final_cohorts.groupby(extend_aquisition_splits).apply(extend_aquisition_data, include_groups=False).reset_index()

In [ ]:
extended_aquisition_data_extended["trial_price_value"] = extended_aquisition_data_extended["trial_price"].str.extract(r'(\d+\.?\d*)')[0].astype(float).fillna(0)
extended_aquisition_data_extended["term_price_value"] = extended_aquisition_data_extended["term_price"].str.extract(r'(\d+\.?\d*)')[0].astype(float).fillna(0)

In [ ]:
extended_aquisition_data_extended.groupby('signup_cohort')['active_users'].sum()

# Merging User Base & Aquisition

In [ ]:
for col in ["active_users", "total_cohort_users"]:
    cohort_df[col] = pd.to_numeric(cohort_df[col], errors="coerce").astype("Int64")
    extended_aquisition_data_extended[col] = (
        pd.to_numeric(extended_aquisition_data_extended[col], errors="coerce").round().astype("Int64")
    )

for col in ["calendar_month", "signup_cohort"]:
    cohort_df[col] = pd.to_datetime(cohort_df[col], errors="coerce").dt.date
    extended_aquisition_data_extended[col] = pd.to_datetime(extended_aquisition_data_extended[col], errors="coerce").dt.date

In [ ]:
extended_aquisition_data_extended["trial_duration_months"] = extended_aquisition_data_extended["trial_duration"].map(cadence_map)

In [ ]:
base_acq_df = pd.concat([cohort_df, extended_aquisition_data_extended], ignore_index=True)

In [ ]:
base_acq_df = base_acq_df.drop('level_8', axis="columns")

In [ ]:
currency_code_map = {
    "UK": "GBP",
    "EUR": "EUR",
    "US": "USD"
}

base_acq_df["payment_currency"] = base_acq_df["region"].map(currency_code_map)

# Applying Retention Curves

In [ ]:
base_acq_retcurves_df = base_acq_df.merge(retention_curves, how='left', on = splits + ["month_index"])

In [ ]:
base_acq_retcurves_df.piecewise_retention_rate = base_acq_retcurves_df.piecewise_retention_rate.fillna(1)

In [ ]:
base_acq_retcurves_df = base_acq_retcurves_df.sort_values(base_splits + ["month_index"])

base_acq_retcurves_df["previous_active_users"] = (
    base_acq_retcurves_df
    .groupby(base_splits, dropna=False)["active_users"]
    .shift(1)
)

def recursive_forecast(group):
    preds = []
    for i, row in group.iterrows():

        # If row is an actual value, keep prediction to be active users
        if row["calendar_month"] <= date(2025,9,1):
            preds.append(row["active_users"])

        elif not preds and row["active_users"] != 0:
            preds.append(row["active_users"])

        elif not preds and row["active_users"] == 0:
            preds.append(0)

        # If a forecast value, multiply the previous predicted value by the piecewise retention rate
        else:
            prev_value = preds[-1] if preds else np.nan
            preds.append(prev_value * row["piecewise_retention_rate"] if pd.notna(prev_value) else np.nan)

    group["predicted_active_users"] = preds

    return group

base_acq_retcurves_forecast_df = base_acq_retcurves_df.groupby(base_splits, group_keys=False, dropna=False).apply(recursive_forecast)

# base_acq_retcurves_forecast_df["is_trialist"] = np.where(
#     base_acq_retcurves_forecast_df["trial_duration_months"] >= base_acq_retcurves_forecast_df["month_index"],
#         True,
#         False
# )

In [ ]:
base_acq_retcurves_forecast_df["is_trialist"] = np.where(
    (
        (base_acq_retcurves_forecast_df["trial_duration_months"] == 0)
        | (base_acq_retcurves_forecast_df["trial_duration_months"] <= (base_acq_retcurves_forecast_df["month_index"]))
    ),
    False,
    True
)


In [ ]:
mask = (base_acq_retcurves_forecast_df["calendar_month"] > date(2025,7,1))

base_acq_retcurves_forecast_df[mask].groupby(["calendar_month"])["predicted_active_users"].sum().reset_index()

In [ ]:
base_acq_retcurves_forecast_df[
    (base_acq_retcurves_forecast_df['package_type'] == 'DIGITAL Subscriber') &
    (base_acq_retcurves_forecast_df['trial_duration_months'] == 6) &
    (base_acq_retcurves_forecast_df['term_cadence'] == "year") &
    (base_acq_retcurves_forecast_df['region'] == 'UK') &
    (base_acq_retcurves_forecast_df['signup_cohort'] == date(2025,3,1))
]

# Currency Conversions

In [ ]:
query = f"""
  SELECT
    Date,
    Currency_Code,
    MAX(GBP_Conversion) AS GBP_Conversion
  FROM `indy-eng.partners.currency_matchtable`
  WHERE
    date <= "{subs_details_rundate}"
  GROUP BY ALL
"""

query_job = client.query(query)
currency_conversion_df = query_job.to_dataframe()
currency_conversion_df["Date"] = pd.to_datetime(currency_conversion_df["Date"]).dt.date

## Fill Conversions

In [ ]:
min_date = date(2025,1,1)
max_date = date(2027,12,31)

date_range = pd.date_range(min_date, max_date, name="Date")

def ffill_currency_conversion(group):

    group = group.sort_values("Date", ascending=True)
    group = group.set_index("Date")
    group = group.reindex(date_range)
    group = group["GBP_Conversion"].ffill()
    group = group.reset_index()

    return group

currency_conversion_extended_df = currency_conversion_df.groupby(["Currency_Code"], group_keys=True).apply(ffill_currency_conversion,include_groups=True).reset_index().drop(["level_1"], axis="columns")
currency_conversion_extended_df["Date"] = pd.to_datetime(currency_conversion_extended_df["Date"]).dt.date

# Offers

In [ ]:
query = f"""
WITH
clean_transactions_df AS (
  SELECT DISTINCT
    DATE(Date) AS transaction_date,
    DATE(TIMESTAMP_TRUNC(Date, MONTH)) AS yearmonth,
    piano_uid,
    tax_type,
    transaction_status,
    term_name,
    term_id,
    subscription_id,
    local_price,
    term_price_value,
    trial_price_value,
    expires,
    tax,
    tax_base,
    tax_rate,
    tax_country,
    term_cadence,
    region,
    package_type,
    customer_type,
    term_price,
    trial_duration,
  FROM
    `indy-eng.reader_revenue_dataform.transactions_log_enhanced`
  WHERE
    customer_type NOT LIKE "Test"
    AND DATE <= "{subs_details_rundate}"
  ),
-- Step 3: Net out refunds from payments to obtain what each uid payed per month
monthly_amount_paid AS (
  SELECT
    transaction_date,
    DATE_TRUNC(transaction_date, MONTH) AS transaction_month,
    piano_uid,
    term_id,
    subscription_id,
    trial_price_value,
    term_price_value,
    term_cadence,
    region,
    package_type,
    customer_type,
    ROUND(SUM(local_price),2) AS monthly_amount_paid
  FROM
    clean_transactions_df
  GROUP BY
    ALL
  ORDER BY
    monthly_amount_paid DESC, piano_uid
  ),
-- Step 4: Join transaction data with subscription_details
subs_details_needed AS (
  SELECT
    piano_uid,
    term_id,
    term_name,
    subscription_id,
    start_date,
    piano_status,
    user_access_expiration_date,
    subscription_trial_end_date
  FROM `indy-eng.reader_revenue_dataform.subscription_details_snapshots_enhanced`

  WHERE
    NOT REGEXP_CONTAINS(LOWER(term_name), r'(app | google | apple)') # As there is no transaction data for app subscriptions, I've removed it from subs
    AND run_date = "{subs_details_rundate}"
),
subs_and_trans AS (
  SELECT
    piano_uid,
    term_id,
    subscription_id,
    start_date,
    subscription_trial_end_date,
    user_access_expiration_date,
    transaction_month,
    term_cadence,
    region,
    package_type,
    customer_type,
    transaction_date,
    trial_price_value,
    term_price_value,
    CASE
      WHEN transaction_date < DATE(subscription_trial_end_date) THEN 'trialist'
      WHEN (transaction_date >= DATE(subscription_trial_end_date) OR subscription_trial_end_date IS NULL) THEN 'non_trialist'
      ELSE 'No tenure'
    END AS tenure,
    monthly_amount_paid
  FROM monthly_amount_paid AS mp
  LEFT JOIN subs_details_needed
  USING(
    piano_uid,
    term_id,
    subscription_id
    )
  ORDER BY
    piano_uid,
    term_id,
    subscription_id,
    transaction_month
)

  SELECT
    COUNT(subscription_id) AS number_of_offers,
    region,
    term_cadence,
    package_type,
    customer_type,
    transaction_month,
    monthly_amount_paid
  FROM
    subs_and_trans
  WHERE
    monthly_amount_paid in (49.00, 20.00, 25.00, 34.99, 40.99, 74.99, 86.99)
    AND transaction_month >= '2024-01-01'
    AND customer_type LIKE "Subscription"
    -- AND term_type='Current Annual'
  GROUP BY
    ALL
  ORDER BY
    number_of_offers DESC
"""

query_job = client.query(query)
offers_df = query_job.to_dataframe()

In [ ]:
offers_df

In [ ]:
offer_splits = ["region", "term_cadence", "package_type", "customer_type", "monthly_amount_paid"]

In [ ]:
import pandas as pd
import numpy as np
from datetime import date
from dateutil.relativedelta import relativedelta
# A change was made in April to higher offer prices and this brought number of offer acceptances down, have limited num_months to reflect this
def last_x_average(group, num_months=5, forecast_months=27):
    # Define date range
    min_date = pd.Timestamp("2024-01-01")
    max_date = pd.Timestamp("2025-09-01")

    # Ensure datetime type
    group["transaction_month"] = pd.to_datetime(group["transaction_month"])

    # Reindex to cover full historical range
    date_range = pd.date_range(min_date, max_date, freq="MS")

    group = (
        group.sort_values("transaction_month")
        .set_index("transaction_month")
        .reindex(date_range)
        .reset_index()
        .rename(columns={"index": "transaction_month"})
    )

    group["key"] = "actuals"
    group["number_of_offers"] = group["number_of_offers"].fillna(0)

    # Iterative forecasting
    forecast_values = []
    last_values = list(group["number_of_offers"].tail(num_months))

    for i in range(forecast_months):
        next_month = max_date + pd.DateOffset(months=i + 1)
        avg_value = np.round(np.mean(last_values[-num_months:]))
        forecast_values.append((next_month, avg_value))
        last_values.append(avg_value)

    # Build forecast DataFrame
    forecast_df = pd.DataFrame(forecast_values, columns=["transaction_month", "number_of_offers"])
    forecast_df["key"] = "forecast"

    # Combine actuals and forecasts
    combined = pd.concat([group, forecast_df], ignore_index=True)
    combined["transaction_month"] = pd.to_datetime(combined["transaction_month"])

    return combined

offers_forecast_df = offers_df.groupby(offer_splits, group_keys=True).apply(last_x_average, include_groups=False)

In [ ]:
offers_forecast_df.groupby('transaction_month')['number_of_offers'].sum()

## Ammortising Offers

In [ ]:
offers_forecast_df = offers_forecast_df.reset_index()
offers_forecast_df.term_cadence.unique()

# Note: Offers can only be yearly so I'm removing any payments that do not have that cadence

offers_forecast_df = offers_forecast_df[offers_forecast_df['term_cadence'] == 'year']

In [ ]:
offers_forecast_df.head(1)

In [ ]:
offers_forecast_df["relevant_cadence_months"] =  12

# Ensure transactions are only amortised throughout the subscription cadence if they're payments - refunds do not get amortised
offers_forecast_df["amortised_summed_local_price"] = offers_forecast_df["monthly_amount_paid"] / offers_forecast_df["relevant_cadence_months"]


offers_forecast_df["transaction_month"] = pd.to_datetime(offers_forecast_df["transaction_month"])

grouping_columns = [
    "term_cadence",
    "region",
    "package_type",
    "customer_type",
    "monthly_amount_paid",
    "relevant_cadence_months"
]

def calculate_user_base(group):
    cadence_interval = 12

    # sort by date
    group = group.sort_values("transaction_month").copy()

    min_date = group["transaction_month"].min()
    max_date = group["transaction_month"].max()

    # Ensure full range of months
    all_dates = pd.date_range(min_date, max_date, freq="MS")

    # Reindex by date to ensure all months exist
    group = (
        group.set_index("transaction_month")
             .reindex(all_dates)
             .rename_axis("transaction_month")
    )

    # Fill missing offers with 0
    group["number_of_offers"] = group["number_of_offers"].fillna(0)

    # Make sure the index is sorted and monotonic
    group = group.sort_index()
    group["user_base"] = group["number_of_offers"].rolling(window=cadence_interval, min_periods=1).sum()

    # Reset back to flat structure
    group = group.reset_index()

    return group


amortised_offers_df = (
    offers_forecast_df
    .groupby(grouping_columns, group_keys=False)
    .apply(calculate_user_base)
    .reset_index(drop=True)
)

amortised_offers_df["amortised_revenue"] = amortised_offers_df["user_base"] * amortised_offers_df["amortised_summed_local_price"]

##Converting Offers Currencies

In [ ]:
currency_code_map = {
    "UK": "GBP",
    "EUR": "EUR",
    "US": "USD"
}
amortised_offers_df['Currency_Code'] = amortised_offers_df['region'].map(currency_code_map)

# Convert 'transaction_month' and 'Date' columns to datetime objects before merging
amortised_offers_df['transaction_month'] = pd.to_datetime(amortised_offers_df['transaction_month'])
currency_conversion_extended_df['Date'] = pd.to_datetime(currency_conversion_extended_df['Date'])


GBP_amortised_offers_df = amortised_offers_df.merge(currency_conversion_extended_df, how="left", left_on=["transaction_month", "Currency_Code"], right_on=["Date", "Currency_Code"]).drop(columns = ['Date', 'Currency_Code'])
GBP_amortised_offers_df['GBP_amortised_revenue'] = GBP_amortised_offers_df['amortised_revenue'] * GBP_amortised_offers_df['GBP_Conversion']

In [ ]:
GBP_amortised_offers_df.head(1)

## Final Dataset

In [ ]:
GBP_amortised_offers_df['transaction_month'] = pd.to_datetime(GBP_amortised_offers_df['transaction_month'])
GBP_amortised_offers_df = GBP_amortised_offers_df[GBP_amortised_offers_df.transaction_month.dt.date >= date(2025,1,1)]

# Final dataset
GBP_amortised_offers_df

# Refunds

In [ ]:
# Adding refunds
query = f"""
  WITH clean_transactions_df AS (
  SELECT
  -- Note: DISTINCT included as transactions log has duplicate identical payments not found in Piano
    DISTINCT
    DATE(Date) AS transaction_date,
    DATE(TIMESTAMP_TRUNC(Date, MONTH)) AS yearmonth,
    piano_uid,
    tax_type,
    transaction_status,
    term_name,
    term_id,
    subscription_id,
    local_price,
    term_price_value,
    trial_price_value,
    expires,
    tax,
    tax_base,
    tax_rate,
    tax_country,
    term_cadence,
    region,
    package_type,
    customer_type,
    term_price,
    trial_cadence,
  FROM
    `indy-eng.reader_revenue_dataform.transactions_log_enhanced`

  WHERE
    -- need to remove tests as these do not represent real subscribers
    LOWER(customer_type) NOT LIKE "test"
    AND DATE <= "{subs_details_rundate}"),


  refunds_2025 AS (
    SELECT DISTINCT
        subscription_id,
        yearmonth,
        term_cadence,
        region,
        package_type,
        customer_type,
        local_price
    FROM clean_transactions_df
    WHERE tax_type='refund'
    AND yearmonth>='2025-01-01'
    AND yearmonth<'2025-10-01'
  )


  SELECT
  ROUND(SUM(local_price),2) AS refund_amount,
  region,
  yearmonth
  FROM refunds_2025
  GROUP BY ALL
  ORDER BY ROUND(SUM(local_price),2)

"""

query_job = client.query(query)
refunds_df = query_job.to_dataframe()

In [ ]:
def generate_refund_forecast(refunds_df, forecast_months=27):

    refunds_df = refunds_df.copy()
    refunds_df['yearmonth'] = pd.PeriodIndex(refunds_df['yearmonth'], freq='M')

    future_refunds = []

    for month_ahead in range(1, forecast_months + 1):
        last_month = refunds_df['yearmonth'].max()
        new_month = last_month + 1  # forecast next month

        # Forecast separately for each region
        for region, group in refunds_df.groupby('region'):
            group = group.sort_values('yearmonth')

            # Take last 6 actual/forecasted months
            recent = group.tail(6)

            if recent.empty:
                continue

            # Simple mean (no weights)
            avg_refund = recent['refund_amount'].mean()

            # Create new forecast row
            new_row = {
                'region': region,
                'yearmonth': new_month,
                'refund_amount': avg_refund
            }

            # Store the forecast record
            future_refunds.append(new_row)

            # Append it so that the next forecast includes it
            refunds_df = pd.concat([refunds_df, pd.DataFrame([new_row])], ignore_index=True)

    return pd.DataFrame(future_refunds)

In [ ]:
forecasted_refunds = generate_refund_forecast(refunds_df)

forecasted_refunds['yearmonth'] = forecasted_refunds['yearmonth'].astype(str) + '-01'
forecasted_refunds['yearmonth'] = pd.to_datetime(forecasted_refunds['yearmonth']).dt.date

forecasted_refunds_full = pd.concat([refunds_df, forecasted_refunds], ignore_index=True)

In [ ]:
forecasted_refunds_full.head(1)

## Converting Refunds Currencies

In [ ]:
forecasted_refunds_full.head(1)

In [ ]:
forecasted_refunds_full['Currency_Code'] = forecasted_refunds_full['region'].map(currency_code_map)

# Convert 'transaction_month' and 'Date' columns to datetime objects before merging
forecasted_refunds_full['yearmonth'] = pd.to_datetime(forecasted_refunds_full['yearmonth'])
currency_conversion_extended_df['Date'] = pd.to_datetime(currency_conversion_extended_df['Date'])


GBP_forecasted_refunds_full = forecasted_refunds_full.merge(currency_conversion_extended_df, how="left", left_on=["yearmonth", "Currency_Code"], right_on=["Date", "Currency_Code"]).drop(columns = ['Date', 'Currency_Code'])
GBP_forecasted_refunds_full['GBP_amortised_revenue'] = GBP_forecasted_refunds_full['refund_amount'] * GBP_forecasted_refunds_full['GBP_Conversion']

## Final Dataset


In [ ]:
# Final Dataset
GBP_forecasted_refunds_full

# Revenue

## Cohorts Merge

In [ ]:
currency_conversion_extended_df['Date'] = pd.to_datetime(currency_conversion_extended_df['Date']).dt.date
base_acq_retcurves_forecast_df['calendar_month'] = pd.to_datetime(base_acq_retcurves_forecast_df['calendar_month']).dt.date

cohort_joined_converted_df = base_acq_retcurves_forecast_df.merge(currency_conversion_extended_df, how="left", left_on=["calendar_month", "payment_currency"], right_on=["Date", "Currency_Code"]).drop(columns=["Date", "Currency_Code"])
cohort_joined_converted_df["GBP_Conversion"] = cohort_joined_converted_df["GBP_Conversion"].fillna(1)

## Revenue Calculation

In [ ]:
cohort_joined_converted_df["paying_amount"] = np.where(
    cohort_joined_converted_df["is_trialist"],
        cohort_joined_converted_df["trial_price_value"],
        cohort_joined_converted_df["term_price_value"]
)

cohort_joined_converted_df["term_cadence_months"] = cohort_joined_converted_df["term_cadence"].map(cadence_map)

cohort_joined_converted_df["current_tenure_cadence"] = np.where(
    cohort_joined_converted_df["is_trialist"],
        cohort_joined_converted_df["trial_duration_months"],
        cohort_joined_converted_df["term_cadence_months"]
)

cohort_joined_converted_df["amortised_paying_amount"] = cohort_joined_converted_df["paying_amount"].astype(float) / cohort_joined_converted_df["current_tenure_cadence"].astype(float)
cohort_joined_converted_df["predicted_amortised_revenue"] = cohort_joined_converted_df["amortised_paying_amount"] * cohort_joined_converted_df["predicted_active_users"]

cohort_joined_converted_df["converted_predicted_amortised_revenue"] = cohort_joined_converted_df["predicted_amortised_revenue"] * cohort_joined_converted_df["GBP_Conversion"]

## Revenue Testing

In [ ]:
dimensions = [
    'package_type',
    'trial_duration_months',
    # 'term_cadence',
    'trial_price_value',
    # 'trial_price',
    # 'term_price',
    'term_price_value',
    'payment_currency',
    'region',
    'signup_cohort',
    # 'trial_duration',
    'calendar_month',
    'month_index',
    'total_cohort_users',
    'retention_curve_rate',
    'piecewise_retention_rate',
    'is_trialist',
    'GBP_Conversion',
    'paying_amount',
    'term_cadence_months',
    'current_tenure_cadence',
    'amortised_paying_amount',
 ]

metrics = [
    'active_users',
    'previous_active_users',
    'predicted_active_users',
    'predicted_amortised_revenue',
    'converted_predicted_amortised_revenue',
]

final_df = cohort_joined_converted_df.groupby(dimensions)[metrics].sum().reset_index()

final_df["trial_price_value"] = final_df["trial_price_value"].astype("float")

In [ ]:
cohort_joined_converted_df[
    (cohort_joined_converted_df['package_type'] == 'DIGITAL Subscriber') &
    (cohort_joined_converted_df['trial_duration_months'] == 6) &
    (cohort_joined_converted_df['trial_price_value'] == "1") &
    (cohort_joined_converted_df['term_cadence_months'] == 12) &
    (cohort_joined_converted_df['region'] == 'EUR') &
    (cohort_joined_converted_df['signup_cohort'] == date(2025,6,1))
]

In [ ]:
final_df[
    (final_df['package_type'] == 'DIGITAL Subscriber') &
    (final_df['trial_duration_months'] == 6) &
    (final_df['trial_price_value'] == 1.00) &
    (final_df['term_cadence_months'] == 12) &
    (final_df['region'] == 'UK') &
    (final_df['signup_cohort'] == date(2025,9,1))
]

In [ ]:
mask = (cohort_joined_converted_df["calendar_month"] == date(2025,9,1))

cohort_joined_converted_df[mask].groupby(["package_type"])[["converted_predicted_amortised_revenue", "predicted_active_users"]].sum().reset_index()

In [ ]:
mask = (cohort_joined_converted_df["calendar_month"] > date(2025,8,1))
mask_2027 = (cohort_joined_converted_df["calendar_month"] >= date(2027,1,1))

cohort_joined_converted_df[mask].groupby(["calendar_month"])[["converted_predicted_amortised_revenue", "predicted_active_users"]].sum().reset_index()

In [ ]:
cohort_joined_converted_df[mask_2027][["converted_predicted_amortised_revenue", "predicted_active_users"]].sum().reset_index()

# KPIs

# Ghsheet Upload

In [ ]:
from google.auth import default
import gspread

creds, _ = default()
gc = gspread.authorize(creds)

forecast_worksheet = gc.open_by_url('https://docs.google.com/spreadsheets/d/1kSjsdMVuyk7weo_ZcaLYn1DbLtrSSOELBlABF24WyWQ/edit?usp=sharing')

## Offers Upload

In [ ]:
GBP_amortised_offers_df.isnull().sum()

In [ ]:
# # Write offers forecasts onto Gsheet
# GBP_amortised_offers_df['transaction_month'] = GBP_amortised_offers_df['transaction_month'].astype(str)

# # Replace NaN values with None to make the DataFrame JSON compliant
# GBP_amortised_offers_df = GBP_amortised_offers_df.replace({np.nan: None})

# forecast_worksheet.worksheet('offers_forecast_raw').clear()
# forecast_worksheet.worksheet('offers_forecast_raw').update([GBP_amortised_offers_df.columns.values.tolist()] + GBP_amortised_offers_df.values.tolist())

## Refunds Upload

In [ ]:
# # Write refunds forecasts onto Gsheet
# GBP_forecasted_refunds_full['yearmonth'] = GBP_forecasted_refunds_full['yearmonth'].astype(str)

# forecast_worksheet.worksheet('refunds_forecast_raw').clear()
# forecast_worksheet.worksheet('refunds_forecast_raw').update([GBP_forecasted_refunds_full.columns.values.tolist()] + GBP_forecasted_refunds_full.values.tolist())




## Standard Upload

In [ ]:
import_df = cohort_joined_converted_df[["package_type", "term_cadence", "trial_price",	"term_price", "region",	"signup_cohort",	"trial_duration", "calendar_month",	"month_index", "predicted_active_users","predicted_amortised_revenue",	"converted_predicted_amortised_revenue"]]

import_df = import_df[import_df.calendar_month >= date(2025,10,1)]

cols_to_convert = import_df.select_dtypes(
    include=["object", "category", "datetime64[ns]", "timedelta64[ns]", "bool"]
).columns

# Convert only those to str
import_df[cols_to_convert] = import_df[cols_to_convert].astype(str)

import_df = import_df.replace({pd.NA: None, np.nan: None})

In [ ]:
forecast_worksheet.worksheet('standard_forecast_fixed').clear()
forecast_worksheet.worksheet('standard_forecast_fixed').update([import_df.columns.values.tolist()] + import_df.values.tolist())

# Base Upload


In [ ]:
base_df = cohort_joined_converted_df[["package_type", "term_cadence", "trial_price",	"term_price", "region",	"signup_cohort",	"trial_duration", "calendar_month",	"month_index", "predicted_active_users","predicted_amortised_revenue",	"converted_predicted_amortised_revenue", "is_trialist"]]
base_df = base_df[base_df['calendar_month'] >= date(2025,10,1)]
base_df['acquisition'] = base_df.loc[base_df['month_index'] == 0, 'predicted_active_users']

base_df['calendar_month'] = base_df['calendar_month'].astype(str)

cols_to_convert = base_df.select_dtypes(
    include=["object", "category", "datetime64[ns]", "timedelta64[ns]", "bool"]
).columns

# Convert only those to str
base_df[cols_to_convert] = base_df[cols_to_convert].astype(str)

# Replace NaN, inf, and -inf values with None to make the DataFrame JSON compliant
base_df = base_df.replace({pd.NA: None, np.nan: None, np.inf: None, -np.inf: None})

forecast_worksheet.worksheet('base_fixed').clear()
forecast_worksheet.worksheet('base_fixed').update([base_df.columns.values.tolist()] + base_df.values.tolist())

In [ ]:
base_df.groupby(['calendar_month'])[['predicted_active_users', 'acquisition','converted_predicted_amortised_revenue']].sum()